In [8]:
%pip install langchain-community langchain-pinecone langchain-openai pinecone -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 8.1 MB/s eta 0:00:00


In [9]:
from google.colab import userdata
import os

os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [33]:
# 데이터 로드
import json

with open('foodDB.json', 'r') as f:
    food_df = json.load(f)

In [34]:
import pandas as pd

df = pd.DataFrame(food_df['COOKRCP01']['row'])
df = df[['RCP_NM', 'RCP_PARTS_DTLS','INFO_ENG']]

df.head()

,RCP_NM,RCP_PARTS_DTLS,INFO_ENG
0,새우 두부 계란찜,"새우두부계란찜\n연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g...",220
1,부추 콩가루 찜,"[1인분]조선부추 50g, 날콩가루 7g(1⅓작은술)\n·양념장 : 저염간장 3g(...",215
2,방울토마토 소박이,"●방울토마토 소박이 : \n방울토마토 150g(5개), 양파 10g(3×1cm), ...",45
3,순두부 사과 소스 오이무침,"●오이무침 :\n오이 70g(1/3개), 다진 땅콩 10g(1큰술)\n●순두부사과 ...",75
4,사과 새우 북엇국,"북엇국\n북어채 25g(15개), 새우 10g(3마리), 사과 30g(1/5개), ...",65


In [35]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# 임베딩할 텍스트(각각 page_content) 생성 - 메뉴이름, 재료
texts = (df['RCP_NM'] + " / " + df['RCP_PARTS_DTLS']).tolist()

# 메타데이터 생성 - 메뉴이름, 칼로리
metadatas = df[['RCP_NM','INFO_ENG']].to_dict(orient='records')

# OpenAI 임베딩 객체 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Pinecone 인덱스명 지정
index_name = "food-index"

batch_size = 10
max_length = 1000  # 필요시 더 짧게 조정

# 텍스트 길이 제한 적용
texts_limited = [t[:max_length] if len(t) > max_length else t for t in texts]

for i in range(0, len(texts_limited), batch_size):
    batch_texts = texts_limited[i:i+batch_size]
    batch_metadatas = metadatas[i:i+batch_size]
    PineconeVectorStore.from_texts(
        texts=batch_texts,
        embedding=embeddings,
        metadatas=batch_metadatas,
        index_name=index_name
    )

In [37]:
# texts를 벡터화 - 이미지 업로드후 임베딩값과 비교시 사용
text_vectors = embeddings.embed_documents(texts)

1. 이미지 임베딩(벡터화)
- 사용자가 이미지를 업로드하면, 해당 이미지를 이미지 임베딩 모델(CLIP, BLIP)로 벡터값으로 변환. streamlit에서 받은 이미지를 벡터화하는 코드 필요.

2. 위에서 얻은 image_vector를 Pinecone에 쿼리하여, 텍스트 임베딩(레시피 등)과 가장 유사한 결과를 검색.
```results = pinecone_index.query(
    vector=image_vector.tolist(),
    top_k=?,
    include_metadata=True
)```

3. 출력된 결과를 기반으로 llm에 질의 및 응답(과정을 반복하며 식단관리 및 운동을 추천)